In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
with open('data-74To95_text.txt', mode='r', errors='ignore') as f:
    text = f.read()

In [3]:
print(text)
# text

id	text	tag
074.jpg	Rechnung	O
074.jpg	Apple	B-ORG
074.jpg	Distribution	I-ORG
074.jpg	International	I-ORG
074.jpg	Ltd.	I-ORG
074.jpg	Hollyhill	O
074.jpg	Industrial	O
074.jpg	Estate	O
074.jpg	Hollyhill,	O
074.jpg	Cork	O
074.jpg	Ireland	O
074.jpg	Ust-IdNr.:	O
074.jpg	DE279463621	O
074.jpg	Rechnungsnummer:	O
074.jpg	DE12ABII34	B-RNUM
074.jpg	Apple	O
074.jpg	Bestellnummer:	O
074.jpg	123456789	O
074.jpg	Rechnungsdatum:	O
074.jpg	01.01.2018	B-DATUM
074.jpg	Fälligkeitsdatum:	O
074.jpg	03.01.2018	O
074.jpg	Kundennummer:	O
074.jpg	1	O
074.jpg	Bestellnummer:	O
074.jpg	ABCD123456/PPAL	O
074.jpg	Zahlungsbedingungen:	O
074.jpg	PAYPAL	O
074.jpg	Bestelldatum:	O
074.jpg	01.01.2018	O
074.jpg	Lieferschein/-datum:	O
074.jpg	0123456789/01.01.2018	O
074.jpg	Artikelnummer	O
074.jpg	|	O
074.jpg	Materialnummer	O
074.jpg	Produktbeschreibung	O
074.jpg	Rechnung	O
074.jpg	an	O
074.jpg	Leon	B-NAME
074.jpg	Babucke	I-NAME
074.jpg	Eichbusch	B-ADRESSE
074.jpg	1	I-ADRESSE
074.jpg	72138	I-ADRESSE
074.jpg	Kirchentellinsf

In [4]:
# split data due to line breaks and Tab separators
data = list(map(lambda x: x.split('\t'), text.split('\n')))

In [5]:
data

[['id', 'text', 'tag'],
 ['074.jpg', 'Rechnung', 'O'],
 ['074.jpg', 'Apple', 'B-ORG'],
 ['074.jpg', 'Distribution', 'I-ORG'],
 ['074.jpg', 'International', 'I-ORG'],
 ['074.jpg', 'Ltd.', 'I-ORG'],
 ['074.jpg', 'Hollyhill', 'O'],
 ['074.jpg', 'Industrial', 'O'],
 ['074.jpg', 'Estate', 'O'],
 ['074.jpg', 'Hollyhill,', 'O'],
 ['074.jpg', 'Cork', 'O'],
 ['074.jpg', 'Ireland', 'O'],
 ['074.jpg', 'Ust-IdNr.:', 'O'],
 ['074.jpg', 'DE279463621', 'O'],
 ['074.jpg', 'Rechnungsnummer:', 'O'],
 ['074.jpg', 'DE12ABII34', 'B-RNUM'],
 ['074.jpg', 'Apple', 'O'],
 ['074.jpg', 'Bestellnummer:', 'O'],
 ['074.jpg', '123456789', 'O'],
 ['074.jpg', 'Rechnungsdatum:', 'O'],
 ['074.jpg', '01.01.2018', 'B-DATUM'],
 ['074.jpg', 'Fälligkeitsdatum:', 'O'],
 ['074.jpg', '03.01.2018', 'O'],
 ['074.jpg', 'Kundennummer:', 'O'],
 ['074.jpg', '1', 'O'],
 ['074.jpg', 'Bestellnummer:', 'O'],
 ['074.jpg', 'ABCD123456/PPAL', 'O'],
 ['074.jpg', 'Zahlungsbedingungen:', 'O'],
 ['074.jpg', 'PAYPAL', 'O'],
 ['074.jpg', 'Bestell

In [6]:
# build your data frame from data above and drop NaN
df = pd.DataFrame(data[1:], columns=data[0])
df.dropna(inplace=True)
df

,id,text,tag
0,074.jpg,Rechnung,O
1,074.jpg,Apple,B-ORG
2,074.jpg,Distribution,I-ORG
3,074.jpg,International,I-ORG
4,074.jpg,Ltd.,I-ORG
...,...,...,...
3643,095.jpg,814584193,O
3644,095.jpg,Seite,O
3645,095.jpg,1,O
3646,095.jpg,von,O


In [7]:
df.head(10)

,id,text,tag
0,074.jpg,Rechnung,O
1,074.jpg,Apple,B-ORG
2,074.jpg,Distribution,I-ORG
3,074.jpg,International,I-ORG
4,074.jpg,Ltd.,I-ORG
5,074.jpg,Hollyhill,O
6,074.jpg,Industrial,O
7,074.jpg,Estate,O
8,074.jpg,"Hollyhill,",O
9,074.jpg,Cork,O


# Text Cleaning
#### . Removing whitespace
#### . Remove unwanted special characters

In [ ]:
# remove white spaces and unnecessary and unwanted punctuations

In [8]:
whitespace = string.whitespace
punctuation = '!#$%&\'*+:;<=>?[\\]^`{|}~'
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
#     text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [9]:
df['text'] = df['text'].apply(cleanText)

In [10]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

C:\Users\Admin\Desktop\InvoiceReader_Project\InvoiceReader\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


# Convert data into Spacy format

In [12]:
# group your data by image id
group = dataClean.groupby(by='id')

In [13]:
# group.groups.keys()

In [14]:
# group.get_group('004.jpg')

In [15]:
# see the result or entities for one invoice
grouparray = group.get_group('074.jpg')[['text', 'tag']].values
content = ''
annotations = {'entities':[]}
start = 0
end = 0
for text, label in grouparray:
    text = str(text)
    stringLength = len(text) + 1
    
    start = end
    end = start + stringLength
    
    if label != 'O':
        annot = (start,end-1,label)
        annotations['entities'].append(annot)
        
    content = content + text + ' '

In [16]:
# grouparray

In [1]:
# content

In [3]:
# annotations

In [4]:
# content.find('359,99') + len('359,99')

In [20]:
# now entities for all of your invoices
invoices = group.groups.keys()

allInvoicesData = []
for invoice in invoices:
    invoiceData = []
    grouparray = group.get_group(invoice)[['text', 'tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '
    
    
    invoiceData = (content,annotations)
    allInvoicesData.append(invoiceData)

In [21]:
allInvoicesData

[('Rechnung Apple Distribution International Ltd. Hollyhill Industrial Estate Hollyhill, Cork Ireland Ust-IdNr. DE279463621 Rechnungsnummer DE12ABII34 Apple Bestellnummer 123456789 Rechnungsdatum 01.01.2018 Fälligkeitsdatum 03.01.2018 Kundennummer 1 Bestellnummer ABCD123456/PPAL Zahlungsbedingungen PAYPAL Bestelldatum 01.01.2018 Lieferschein/-datum 0123456789/01.01.2018 Artikelnummer Materialnummer Produktbeschreibung Rechnung an Leon Babucke Eichbusch 1 72138 Kirchentellinsfurt Germany Versand an 1 Leon Babucke Eichbusch 1 72138 Kirchentellinsfurt Germany Anzahl Preis je Einheit Summe Steuersatz (exkl. MwSt.) (exkl. MwSt.) Lieferbedingungen/Incoterms CIP - Frachtgebühr, Versicherung gezahlt an 123456 LLLXXLL/L Apple MacBook Air Ret. """13""""""" -2020 APPLE WEEE REGISTRATION NUMBER DE 71048874 Steuernummer 17/070/28455 Dieser Rechnungsbetrag wurde beglichen mit PAYPAL Bei Fragen zu dieser Rechnung oder zur Auslieferung der Ware wenden Sie sich bitte an unsere Kundendienstabteilung Ruf

# Splitting the Data into Training and Testing Set

In [22]:
import random

In [23]:
random.shuffle(allInvoicesData)

In [24]:
len(allInvoicesData)

22

In [25]:
# divide data 1/3 or 1/4 
# 1 is --> TestData
# in our case we used 22 invoices of 6 different types so 8 for test purposes
# allInvoicesData[:14]
allInvoicesData[14:]

[('HP Deutschland GmbH 71034 BÖBLINGEN Rechnung an Luise Abas Friedenstraße 6 68753 Waghäusel Deutschland Verkauf an Luise Abas Friedenstraße 6 68753 Waghäusel Deutschland Versand an Luise Abas Attn Luise Abas Friedenstraße 6 68753 Waghäusel Deutschland Internal Post DE Rechnung Original 18H494244444 TZXB36355264 HP Auftragsnummer Web Auftragsnummer TZXB36355264 14.05.2021 552037807 Kundenauftragsnummer Kundenauftragsdatum Kundennummer Vertragsnummer Name des Einkäufers Ihre UST-ID HP UST-ID Rechnungsempfänger DE454491700 Tel. e-mail DESTORE.POSTSALES@HP.COM Vielen Dank - der Rechnungsbetrag wurde Ihrer Kreditkarte bzw. Ihrem PayPal-Konto belastet Kommentar Deal ID/Version 43207618 Pos. Menge Produkt-Nr. Beschreibung Preis Summe 100 1 958K8ST Apple MacBook Air 0,83 EUR 0,83 HS. No 8471300000 Lieferschein Nr./Lieferdatum 142726129 / 16.05.2021 Pack-ID VIV30U7Z62 Seriennummer 1 ABD ABD-Deutsche Lokalisierung Versandkosten exkl. MwSt. EUR 0 Zwischensumme EUR 0,83 DE 19 USt. EUR 0,16 GESAM

In [26]:
TrainData = allInvoicesData[:14]
TestData = allInvoicesData[14:]

In [ ]:
# now save your data in pickle format

In [27]:
import pickle

In [28]:
pickle.dump(TrainData,open('./data/TrainData.pickle', mode='wb'))
pickle.dump(TestData,open('./data/TestData.pickle', mode='wb'))